# 1st

## Model Explore

In [ ]:
!pip install -q transformers accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
notebook_login()

## more model

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Load model with 8-bit precision (needs less memory)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,  # For lower RAM usage (needs `bitsandbytes`)
    torch_dtype=torch.float16,
)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
def answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # output = model.generate(**inputs, max_new_tokens=200)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        eos_token_id=tokenizer.eos_token_id)
    # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = " I have knee pain which medicine should i take?"
answer(prompt)

" I have knee pain which medicine should i take?\nWhat type of pain is it?\nHow long has it been going on?\nWhat makes the pain worse?\nDo you have any other symptoms?\nDo you have any history of injury or illness?\nWhat do you do about it?\nDo you take any medications for it?\nWhat other medications do you take?\nWhat is your job?\nWhat do you do for fun?\nWhat is your favorite food?\nDo you have any other questions?\nIf so, ask them.\nI'll explain more about these questions later in the book.\n11\nThe first thing you need to do is to stop worrying about the pain. It is not as bad as you think"

In [ ]:
prompt = "I have headache which medicine should i take?"
answer(prompt)

"I have headache which medicine should i take?\nAnswers from experts on medicine for headache. First: If you've had a headache for 2 weeks, it's not a headache, it's a brain tumor. I would suggest you get to an ER right away. Second: It depends on the type of headache you have. The best medicine is to find the cause. Third: I have a headache that lasts 10 days. I took aspirin and it did not help. I took ibuprofen and it did not help. What else can I take?\nWhat is the best medicine for a headache?\nI have a headache. What is the best medicine for it?\nCan I"